In [1]:
# Mounting
tiers=["bronze","silver","gold"]
adls_paths = {tier: f"abfss://{tier}@synonlypunya.dfs.core.windows.net/" for tier in tiers}
bronze_adls=adls_paths['bronze']
silver_adls=adls_paths['silver']
gold_adls=adls_paths['gold']

StatementMeta(synapseonly, 0, 2, Finished, Available, Finished)

In [2]:
print(bronze_adls)

StatementMeta(synapseonly, 0, 3, Finished, Available, Finished)

abfss://bronze@synonlypunya.dfs.core.windows.net/


In [5]:
files=mssparkutils.fs.ls(bronze_adls)

for file in files:

    print(file.name,file.isDir,file.isFile,file.path,file.size,file.modifyTime)


StatementMeta(synapseonly, 0, 6, Finished, Available, Finished)

a


In [6]:
import requests
import json
from datetime import date, timedelta

StatementMeta(synapseonly, 0, 7, Finished, Available, Finished)

In [8]:
#Remove this before running data factory pipeline
start_date=date.today()-timedelta(1)
end_date=date.today()

StatementMeta(synapseonly, 0, 9, Finished, Available, Finished)

In [9]:
start_date, end_date

StatementMeta(synapseonly, 0, 10, Finished, Available, Finished)

(datetime.date(2025, 1, 29), datetime.date(2025, 1, 30))

StatementMeta(synapseonly, 0, 15, Finished, Available, Finished)

ModuleNotFoundError: No module named 'pyspark.dbutils'

In [16]:
# Construct the API URL with start and end dates provided by Data Factory, formatted for geojson output.
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"

try:
    # Make the GET request to fetch data
    response = requests.get(url)

    # Check if the request was successful
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    data = response.json().get('features', [])

    if not data:
        print("No data returned for the specified date range.")
    else:
        # Specify the ADLS path
        file_path = f"{bronze_adls}/{start_date}_earthquake_data.json"
        
        # Save the JSON data
        json_data = json.dumps(data, indent=4)

        #creating an RDD with json string and parallelize
        rdd= spark.sparkContext.parallelize([json_data])
        
        # creating DF from RDD
        df=spark.read.json(rdd)
        df.write.mode("overwrite").json(file_path)

except requests.exceptions.RequestException as e:
    print(f"Error fetching data from API: {e}")

StatementMeta(synapseonly, 0, 17, Finished, Available, Finished)

In [17]:
data[0]

StatementMeta(synapseonly, 0, 18, Finished, Available, Finished)

{'type': 'Feature',
 'properties': {'mag': 0.68,
  'place': '14 km N of Borrego Springs, CA',
  'time': 1738195018480,
  'updated': 1738195304790,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ci40174562',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci40174562&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 7,
  'net': 'ci',
  'code': '40174562',
  'ids': ',ci40174562,',
  'sources': ',ci,',
  'types': ',nearby-cities,origin,phase-data,scitech-link,',
  'nst': 33,
  'dmin': 0.07217,
  'rms': 0.2,
  'gap': 65,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 0.7 - 14 km N of Borrego Springs, CA'},
 'geometry': {'type': 'Point',
  'coordinates': [-116.3943333, 33.3758333, 9.55]},
 'id': 'ci40174562'}

In [18]:
import json

output_data={
    "start_date": start_date.isoformat(),
    "bronze_adls": bronze_adls,
    "silver_adls": silver_adls,
    "gold_adls": gold_adls
}

bronze_output= json.dumps(output_data)
mssparkutils.notebook.exit(bronze_output)

StatementMeta(synapseonly, 0, 19, Finished, Available, Finished)

ExitValue: {"start_date": "2025-01-29", "bronze_adls": "abfss://bronze@synonlypunya.dfs.core.windows.net/", "silver_adls": "abfss://silver@synonlypunya.dfs.core.windows.net/", "gold_adls": "abfss://gold@synonlypunya.dfs.core.windows.net/"}